## Basic example: prompt + model + output parser

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_BASE"] = "http://ai-ol.sns.sohu.com/v1"
os.environ["OPENAI_API_KEY"] = "0d6e513ab4cb43dd911560b08aa69aec"

prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they always bring the scoop!"

In [6]:
# groq
# from groq import Groq
from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"] = "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"

prompt = ChatPromptTemplate.from_template(
    "给我讲一个关于{topic}的小笑话"
)
model = ChatGroq()  # model="mixtral-8x7b-32768"
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "大学"})

"为什么大学生总是睡得那么香？\n\n因为他们学的是hard-ware， dorm-room里又那么 warm-hearted！(hardware refers to the subjects they study, while dorm-room refers to their bedroom)\n\n(Note: This joke is a play on words in Chinese, and it doesn't translate well into English.)"

## RAG Search Example

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

os.environ["OPENAI_API_BASE"] = "http://ai-ol.sns.sohu.com/v1"
os.environ["OPENAI_API_KEY"] = "0d6e513ab4cb43dd911560b08aa69aec"

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriver = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Qustion: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retriever = RunnableParallel(
    {"context": retriver, "question": RunnablePassthrough()}
)
chain = setup_and_retriever | prompt | model | output_parser

chain.invoke("where dis harrison work?")

/home/vscode/.local/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Harrison worked at Kensho.'